### Building better prompts

### Import library

In [13]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import requests

In [14]:
# Load environment variables
load_dotenv()
api_key = os.getenv('PPLX_API_KEY')

### Authenticate

In [21]:
# Set the endpoint URL
url = 'https://api.perplexity.ai/v1/completions'

# Set the headers for the API request
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

base_url = 'https://api.perplexity.ai'

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

# Make an API request
response = requests.post(
    f'{base_url}/v1/completions',
    headers=headers,
    json={
        'model': 'claude-3',
        'prompt': 'Hello Claude! How are you doing today?',
        'max_tokens': 100,
        'temperature': 0.7
    }
)

# Check the response status code
if response.status_code == 200:
    result = response.json()
    print(result['completion'])
else:
    print(f'Request failed with status code {response.status_code}')

Request failed with status code 404


### Create API request with data

In [22]:
import requests

YOUR_API_KEY = api_key
base_url = "https://api.perplexity.ai"

headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json",
}

messages = [
    {
        "role": "system",
        "content": (
            "You are an artificial intelligence assistant and you need to "
            "engage in a helpful, detailed, polite conversation with a user."
        ),
    },
    {
        "role": "user",
        "content": "How many stars are in the universe?",
    },
]

# chat completion without streaming
response = requests.post(
    f"{base_url}/v1/completions",
    headers=headers,
    json={
        "model": "llama-3-sonar-large-32k-online",
        "messages": messages,
    },
)

if response.status_code == 200:
    result = response.json()
    print(result["completion"])
else:
    print(f"Request failed with status code {response.status_code}")

Request failed with status code 404


In [24]:
from openai import OpenAI

YOUR_API_KEY = api_key

messages = [
    {
        "role": "system",
        "content": (
            "You are an artificial intelligence assistant and you need to "
            "engage in a helpful, detailed, polite conversation with a user."
        ),
    },
    {
        "role": "user",
        "content": "How many stars are in the universe?",
    },
]

client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

# chat completion without streaming
response = client.chat.completions.create(
    model="llama-3-sonar-large-32k-online",
    messages=messages,
)
print(response)

ChatCompletion(id='d6ce9960-ceca-4e3d-a046-271e40dfb8a1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The number of stars in the universe is a mind-boggling topic that has fascinated humans for centuries. While we can't give an exact count, astronomers have made various estimates based on observations and calculations.\n\n**The Milky Way Galaxy**\n-------------------------\n\nOur home galaxy, the Milky Way, is estimated to contain approximately **400 billion stars**. This number is significant, but it's just a small fraction of the total number of stars in the universe.\n\n**Galaxies in the Universe**\n---------------------------\n\nThe number of galaxies in the universe is estimated to be between **6 and 20 trillion**. This vast number is based on observations and extrapolations of what lies beyond our current capabilities to observe.\n\n**Estimating the Total Number of Stars**\n--------------------------------------\n\nA simple